# Simple visual

Take the data from LanceDB and form up some dataframes to load up KuzuDB with.  Then do a few simple visualizations to see relations.

Next steps:
- https://docs.kuzudb.com/extensions/vector/
- https://docs.kuzudb.com/extensions/full-text-search/


In [1]:
import kuzu
from yfiles_jupyter_graphs_for_kuzu import KuzuGraphWidget
import lancedb
import pandas as pd
import hashlib
from sentence_transformers import SentenceTransformer


In [2]:
db = lancedb.connect("../lancedb")

sources_df = pd.DataFrame(db.open_table("sources").to_pandas())
claims_df = pd.DataFrame(db.open_table("claims").to_pandas())
entities_df = pd.DataFrame(db.open_table("entities").to_pandas())


In [3]:
print(sources_df.columns.values)
print("---------------------------------")
print(claims_df.columns.values)
print("---------------------------------")
print(entities_df.columns.values)


['filename' 'id' 'location' 'markdown']
---------------------------------
['filename' 'node_name' 'index' 'text']
---------------------------------
['start' 'end' 'text' 'label' 'score' 'filename' 'nodename' 'index']


Based on the above, I could make "from" and "to" columns in all of these.  Then also make a node_id and name for all of them as well.

## Look at Sources

No need to modify this table

In [4]:
sources_df

,filename,id,location,markdown
0,2202.05901v2.pdf,2c494b74cd387935028898cf4d998e1c,file://stores/inputs/2202.05901v2.pdf,## Page 1\n\nIdentiﬁcation of Flux Rope Orient...
1,2409.09230v1.pdf,824512f309bf431858125747565b05f6,file://stores/inputs/2409.09230v1.pdf,## Page 1\n\nSolar Physics\nDOI: 10.1007/ ••••...
2,source3,08260e29029420a6ba81dcc7cd7dc033,https://gc.copernicus.org/articles/2/55/2019/,![](https://contentmanager.copernicus.org/8009...


## Update the claims

Need to make an ID, would likely do this in the generation phase in the end

In [5]:
claims_df['composite_id'] = claims_df['filename'] + '_' + claims_df['node_name'] + '_' + claims_df['index'].astype(str)

In [6]:
claims_df

,filename,node_name,index,text,composite_id
0,2202.05901v2.pdf,Cardinal,0,This study demonstrates a convolutional neural...,2202.05901v2.pdf_Cardinal_0
1,2202.05901v2.pdf,Cardinal,1,The neural networks trained with full and part...,2202.05901v2.pdf_Cardinal_1
2,2202.05901v2.pdf,Cardinal,2,The accuracy of the neural network in predicti...,2202.05901v2.pdf_Cardinal_2
3,2202.05901v2.pdf,Supporting,0,Focusing on the convolutional neural network's...,2202.05901v2.pdf_Supporting_0
4,2202.05901v2.pdf,Supporting,1,"98,000 synthetic flux ropes of varying orienta...",2202.05901v2.pdf_Supporting_1
...,...,...,...,...,...
71,source3,Cardinal,2,Scientometric analysis of keyword usage can re...,source3_Cardinal_2
72,source3,Supporting,0,The data was collected from 17 years of AGU Fa...,source3_Supporting_0
73,source3,Supporting,1,Network density and connected components were ...,source3_Supporting_1
74,source3,Supporting,2,Keyword analysis was performed using AGU's key...,source3_Supporting_2


## Update entities

Make some ids for use in building the network later.

Again, this is likely something we would want to do at generation time, not a post process event

In [7]:
entities_df['text_md5'] = entities_df['text'].apply(lambda x: hashlib.md5(str(x).encode()).hexdigest())

In [8]:
entities_df['composite_id'] = entities_df['filename'] + '_' + entities_df['nodename'] + '_' + entities_df['index'].astype(str)

In [9]:
entities_df

,start,end,text,label,score,filename,nodename,index,text_md5,composite_id
0,26,54,convolutional neural network,Model,0.802835,2202.05901v2.pdf,Cardinal,0,74c081a322a1b6000c9a79efe486c029,2202.05901v2.pdf_Cardinal_0
1,58,61,CNN,Model,0.624678,2202.05901v2.pdf,Cardinal,0,2435a45b85628172c5a47122144a7c67,2202.05901v2.pdf_Cardinal_0
2,116,127,orientation,Metric,0.650835,2202.05901v2.pdf,Cardinal,0,da1639422ad8f355d2371428471379b5,2202.05901v2.pdf_Cardinal_0
3,129,145,impact parameter,Metric,0.847192,2202.05901v2.pdf,Cardinal,0,8abe3c2e3dce9d23c0dcb3a60be1d595,2202.05901v2.pdf_Cardinal_0
4,151,160,chirality,Metric,0.788682,2202.05901v2.pdf,Cardinal,0,a0cbfd960a9ddd3fa2ad063f48d1d4dd,2202.05901v2.pdf_Cardinal_0
...,...,...,...,...,...,...,...,...,...,...
158,183,203,connected components,System,0.633513,source3,Supporting,1,4f075a5c6b893a805fb941a4db775c4a,source3_Supporting_1
159,0,7,Keyword,Keyword,0.769558,source3,Supporting,2,220f3d3750583b9db48568a0b5b9a5f1,source3_Supporting_2
160,37,40,AGU,Journal,0.525274,source3,Supporting,2,75dc08b63d6e7e50cf5fa69ea6b03193,source3_Supporting_2
161,4,21,analysis software,Software,0.767122,source3,Supporting,3,fb3cda0a5978a3d4822436ebd6ccfd12,source3_Supporting_3


## Build the nodes and relations from Sources, Claims and Entities

Note the fact I have both an entities table and a nanopubs table

In [10]:
# Create the relations dataframe from claims_df
relations_df = pd.concat([
    pd.DataFrame({
        'from': claims_df['filename'],
        'to': claims_df['composite_id']
    }),
    pd.DataFrame({
        'from': entities_df['composite_id'],
        'to': entities_df['text']
    })
])


In [11]:
# TODO  bring in the "text" from claims and .. add columns "description" and desc_embedding
nodes = pd.DataFrame(pd.concat([
    pd.concat([sources_df['filename'].rename('id'), pd.Series(['source'] * len(sources_df), name='type')], axis=1),
    pd.concat([claims_df['composite_id'].rename('id'), pd.Series(['claim'] * len(claims_df), name='type')], axis=1),
    pd.concat([entities_df['text'].rename('id'), pd.Series(['entities'] * len(entities_df), name='type')], axis=1)
])).drop_duplicates('id')


In [12]:
# load the model, this can take a while the first time
model = SentenceTransformer("all-MiniLM-L6-v2")

/home/fils/scratch/biohack/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
# Open a new in-memory database
db = kuzu.Database()
conn = kuzu.Connection(db)
# conn.execute("INSTALL vector; LOAD vector;")


In [14]:
# # MEW APPROACH
# # alternate node table(s) approach
# conn.execute("CREATE NODE TABLE SOURCES(id STRING PRIMARY KEY, filename string)")
# kuzu_sources_df = pd.DataFrame({
#     'id': sources_df['id'],
#     'filename': sources_df['filename']
# })
# conn.execute("COPY SOURCES FROM kuzu_sources_df (ignore_errors=true)")
#
# conn.execute("CREATE NODE TABLE CLAIMS(id STRING PRIMARY KEY, text string, nodename string, filename string)")
# kuzu_sources_df = pd.DataFrame({
#     'id': claims_df['composite_id'],
#     'text': claims_df['text'],
#     'nodename': claims_df['node_name'],
#     'filename': claims_df['filename']
# })
# conn.execute("COPY CLAIMS FROM kuzu_sources_df (ignore_errors=true)")
#
# conn.execute("CREATE NODE TABLE ENTITIES(id STRING PRIMARY KEY, text string, label string, nodename string, filename string)")
# kuzu_sources_df = pd.DataFrame({
#     'id': entities_df['composite_id'],
#     'text': entities_df['text'],
#     'label': entities_df['label'],
#     'nodename': entities_df['nodename'],
#     'filename': claims_df['filename']
# })
# conn.execute("COPY ENTITIES FROM kuzu_sources_df (ignore_errors=true)")


In [15]:
# OLD APPROACH
# Creates a Person node table with name as the primary key

conn.execute("CREATE NODE TABLE Claim(id STRING PRIMARY KEY, type string)") # add in description and desc_embedding

# Enable the `ignore_errors` parameter below to ignore the erroneous rows
conn.execute("COPY Claim FROM nodes (ignore_errors=true)")

In [16]:
conn.execute("CREATE REL TABLE IF NOT EXISTS rels( FROM Claim TO Claim)")

In [17]:
res = conn.execute("COPY rels FROM relations_df")

In [18]:
# Create a widget instance using the existing connection
g = KuzuGraphWidget(conn)

def get_node_color(node):
    node_type = node["properties"]["type"]
    if node_type == "source":
        return "blue"  # Or any color you prefer for source
    elif node_type == "claim":
        return "green" # Or any color you prefer for claim
    elif node_type == "entities":
        return "purple" # Or any color you prefer for entity
    else:
        return "gray"  # Default color for other types

g.add_node_configuration(
    "Claim",  # You might want to change this if the configuration is not just for "Person" nodes
    color=lambda node: get_node_color(node),  # type: ignore
    text=lambda node: {  # type: ignore
        "text": node["properties"]["type"],
        "position": "south",
    }
)
# set up configuration for the graph
# Custom configuration for nodes
# g.add_node_configuration(
#     "Claim",
#     color="red",   # type: ignore
#      text= lambda node : {   # type: ignore
#          "text": node["properties"]["type"],
#          "position": "south",
#     }
# )

# Display the entire graph

In [19]:
g.show_cypher("MATCH (a)-[b]->(c) RETURN *")


GraphWidget(layout=Layout(height='800px', width='100%'))

In [20]:
# g.show_cypher("MATCH (a)-[]->(intermediate_node {type: 'claim'})-[]->(c) RETURN *")
# g.show_cypher("MATCH conn_path = (a)-[]->(intermediate_node {type: 'claim'})-[]->(c) RETURN a, c, conn_path")
# g.show_cypher("MATCH conn_path = (a)-[]->(intermediate_node {type: 'claim'})-[]->(c) RETURN a, c, conn_path")
# MATCH p = (s {type: 'source'})-[*]-(e {type: 'entities'})
# RETURN p

In [21]:
# g.show_cypher("MATCH p = (start_node {type: 'source'})-[]->(intermediate_node)-[]->(end_node {type: 'entities'}) RETURN p")
g.show_cypher("MATCH p = (start_node {type: 'source'})-[]->(intermediate_node)-[]->(end_node {type: 'entities'}) RETURN start_node, end_node, p")

GraphWidget(layout=Layout(height='800px', width='100%'))